In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [ ]:
rawdata = pd.read_csv('D:/data cafe/MyProject_datacafe/training set/data_for_train.csv', delimiter=',', quotechar='"')
rawdata_np = rawdata.as_matrix()

In [ ]:
tokenized_tag = []
for i in rawdata_np [ : ,2]:
    tags = i.replace(' ','').replace('#','').replace('##','').split(',')
    for tag in tags :
        if tag not in tokenized_tag:
            tokenized_tag.append(tag)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)
mlb.fit([tokenized_tag])

In [ ]:
# topic tags
topic_tokenized_tag = []
id_list = []
for id,strtag in rawdata_np [ : ,[1,2]]:
    if id not in id_list:
        id_list.append(id)
        tags = strtag.replace(' ','').replace('#','').replace('##','').split(',')
        topic_tokenized_tag.append(tags)

In [ ]:
# user tags 
user_tag = {}
for id,strtag in rawdata_np [ : ,[0,2]]:
    tags = strtag.replace(' ','').replace('#','').replace('##','').split(',')
    if id not in user_tag:
        user_tag[id] = tags
    else:
        for tag in tags:
            if tag not in user_tag[id]:
                user_tag[id].append(tag)

user_tokenized_tag = []
user_id = []
for uid in user_tag:
    user_id.append(uid)
    user_tokenized_tag.append(user_tag[uid])

In [ ]:
# mlb transform
topic_matrix = mlb.transform(topic_tokenized_tag)
user_matrix = mlb.transform(user_tokenized_tag)

In [ ]:
#user dict
user_topic_read = pd.read_csv('D:/data cafe/MyProject_datacafe/data_set/user_dict.csv', delimiter=',', quotechar='"')
user_topic_read_np = user_topic_read.as_matrix()
user_topic_read_dict = {}
for i in user_topic_read_np:
    if user_topic_read_dict.get(i[0]):
        user_topic_read_dict[i[0]].append(i[1])
    else:
        user_topic_read_dict[i[0]] = [i[1]]
# print(user_topic_read_dict)

In [ ]:
def distance(sparse_p1, sparse_p2):
    p1 = sparse_p1.toarray()
    p2 = sparse_p2.toarray()
    if len(p1) == len(p2):
        distance = np.sum(np.logical_xor(p1, p2))
        return distance
    else:
        print("Those 2 points have different dimension")

In [ ]:
recommendation_result = []
for u_ind, user in enumerate(user_matrix):
    min_distance = 99999
    recommended_topic_id = 99999
    for t_ind, topic in enumerate(topic_matrix):
        dist = distance(user, topic)
#         dist = distance.euclidean(user.toarray(),topic.toarray())
        if dist < min_distance:
            min_distance = dist
            recommended_topic_id = id_list[t_ind]
    result.append(recommended_topic_id)
    if u_ind + 1 % 1000 == 0:
        print("Running on user index :",u_ind+1)
    print(u_ind)

In [ ]:
#open file 
import json
d = dict(zip(user_id[:1145], recommendation_result))
json.dump(d, open('recommendation_for_user_dict.json', 'w+'))
with open('D:/data cafe/recommendation/recommendation_for_user_dict.json') as recommendation_for_user_dict:    
    recommendation_for_user_dict = json.load(recommendation_for_user_dict)

In [ ]:
#check duplicate
recommendation_for_user_dict_not_dup={}
for user_id in recommendation_for_user_dict:
    a = recommendation_for_user_dict[user_id]
    b = user_topic_read_dict[int(user_id)]
    if a not in b:
        recommendation_for_user_dict_not_dup[user_id] = recommendation_for_user_dict[user_id]

In [ ]:
# test set
test_set_read = pd.read_csv('D:/data cafe/MyProject_datacafe/test set/data_for_test.csv', delimiter=',', quotechar='"')
test_set_np = test_set_read.as_matrix()
test_set_dict = {}
for i in test_set_np:
    if test_set_dict.get(i[0]):
        test_set_dict[i[0]].append(i[1])
    else:
        test_set_dict[i[0]] = [i[1]]

In [ ]:
# evaluate model
evaluation_recomendation={}
for user_id in recommendation_for_user_dict_not_dup:
    a = recommendation_for_user_dict_not_dup[user_id]
    b = test_set_dict[int(user_id)]
    if a in b:
        evaluation_recomendation[user_id] = recommendation_for_user_dict_not_dup[user_id]